In [ ]:
import os
import numpy as np
import pandas as pd

RUTA_TEMPERATURA = "/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/temperatura.csv"
RUTA_DEMANDA = "/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/demanda.csv"

Esta parte se utilizara para preprocesar los datos de entrada y generar un dataset unico para ir probando diversos modelos.

* Los datos de demanda son extraidos de las estadisticas de CAMMESA
* Los datos de temperatura son extraidos del SMN

El largo de estos datos comprende desde `01-01-2021` al `31-01-2024`. Con un paso entre muestra y muestra de una (1) hora.

> Como objetivo nos vamos a plantear predecir solo el mes de enero, `01-01-2024` al `31-01-2024`.



In [ ]:
# Leer demanda
def prepocesar_demanda(df):
    df['fecha'] = pd.to_datetime(df['fecha'])   # A objeto fecha
    df = df.set_index('fecha')                  # Indexar por fecha
    df = df[df['provincia'] == "TOTAL"]         # Demanda del SADI
    df = df[['demanda', 'no habil']]            # Selecciono columnas
    df = df.rename(
        {'no habil': 'no_habil'}, axis=1)       # Elimino espacio para usar dot sintax
    return df

demanda = pd.read_csv(RUTA_DEMANDA)
demanda = prepocesar_demanda(demanda)
demanda.head(5)

,demanda,no_habil
fecha,,
2021-01-01 01:00:00,12329.594,True
2021-01-01 02:00:00,12097.918,True
2021-01-01 03:00:00,11805.739,True
2021-01-01 04:00:00,11447.416,True
2021-01-01 05:00:00,11143.879,True


In [ ]:
# Leer Temperatura
def prepocesar_temperatura(df):
    df['fecha'] = pd.to_datetime(df['fecha'])       # A objeto fecha
    df = df.set_index('fecha')                      # Indexar por fecha

    data.temp = data.temp.fillna(method='bfill')    # Lleno valores faltantes con el dato anterior

    # Anoto los dias calidos y frios
    df['es_dia_calido'] = (df['temp'] > df['Temperatura máxima (°C)']).astype(int)
    df['es_dia_frio'] = (df['temp'] < df['Temperatura mínima (°C)']).astype(int)

    df = df.groupby(['PROVINCIA', 'fecha']).mean(numeric_only=True)  # Agrupo por region
    df = df.loc['CAPITAL FEDERAL']                                   # Selecciono GBA por ser centro de demanda

    df = df[['temp', 'es_dia_calido', 'es_dia_frio']]   # Selecciono columnas

    return df

temperatura = pd.read_csv(RUTA_TEMPERATURA)
temperatura = prepocesar_temperatura(temperatura)
temperatura.tail()

,temp,es_dia_calido,es_dia_frio
fecha,,,
2024-01-31 19:00:00,31.80,1.0,0.0
2024-01-31 20:00:00,30.85,1.0,0.0
2024-01-31 21:00:00,30.25,1.0,0.0
2024-01-31 22:00:00,29.10,0.5,0.0
2024-01-31 23:00:00,28.70,0.5,0.0


In [ ]:
# Armar Dataset
data = demanda.join(temperatura, how="left")                           # Uno ambos datos
data = data.loc["2021-01-01":"2024-01-31"]                             # Ajusto longitud de datos
data = data.resample(rule='H', closed='left', label ='right').mean()   # Verifico que queden cada una hora
data.to_csv("/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/dataset.csv")
data.tail()

,demanda,no_habil,temp,es_dia_calido,es_dia_frio
fecha,,,,,
2024-01-31 20:00:00,23976.806,0.0,31.80,1.0,0.0
2024-01-31 21:00:00,22933.568,0.0,30.85,1.0,0.0
2024-01-31 22:00:00,23470.519,0.0,30.25,1.0,0.0
2024-01-31 23:00:00,24386.953,0.0,29.10,0.5,0.0
2024-02-01 00:00:00,24334.169,0.0,28.70,0.5,0.0


El dataset formado sera este para evitar variaciones en los resultados,

Se guarda en el archivo `dataset.csv` y se podra cargar a traves del comando `pd.read_csv("Datasets/dataset.csv")`. Cabe destacar que el notebook debe estar sobre la raiz del directorio que contiene dicha carpeta.

# Preparacion del entorno de ejecución y de los datos

In [ ]:
# instalo skforecast
!pip install skforecast

In [ ]:
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster

In [ ]:
# se definen los intervalos de entranamiento y pronostico
end_train = '2023-12-31 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Train dates      : 2021-01-01 02:00:00 --- 2023-12-31 23:00:00  (n=26278)
Test dates       : 2024-01-01 00:00:00 --- 2024-02-01 00:00:00  (n=745)


In [ ]:
# se ilustra los intervalos tomados
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['demanda'], name='Entrenamiento'))
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['demanda'], name='A Pronosticar'))
fig.update_layout(
    title  = 'Demanda',
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()

# Modelo minimo

En esta seccion se muestra la forma de preparacion de un modelo ingenuo que pronostica la demanda como el mismo valor del dia anterior.

Se va a mostrar la preparacion de los datos y el uso de `skforecast`.

In [ ]:
# se selecciona el pronosticador
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=1),
                 n_offsets = 1
             )

In [ ]:
# entrenarlo y obtener mediciones (NO DEVUELVE EL PRONOSTICADOR YA ENTRENADO)
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 3123072.71


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Demanda y pronostico",
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()

# Modelo lineal

Se puede hacer una regresion lineal mirando 24 horas hacia atras. Para esto se
utilizara el regresor lineal de `sklearn` aplicado a un modelo autoregresivo (Lo que se mostro en el [Excel](https://docs.google.com/spreadsheets/d/1Mw4F2pu4FfPlJbHOpXjDxpCe3VhlOnNX/edit?usp=drive_link&ouid=103999790129601276141&rtpof=true&sd=true)).

In [ ]:
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoreg import ForecasterAutoreg

forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24                         # cuanto mira hacia atras
)

In [ ]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 2289726.72


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Demanda y pronostico",
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()

# Agregar informacion extra (temperatura)
El agregado de variables externas (exogenas), permite tener un mejor resultado al tener en cuenta las causas de la variabilidad de la señal a pronosticar.

En este caso la temperatura influye directamente sobre la demanda. Por este motivo utilizaremos como primera instancia el valor de temperatura en el momento de la predicción.

---

Se va a utilizar el modelo lineal anterior para mostrar como se pueden sumar los datos.

In [ ]:
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24                         # cuanto mira hacia atras
)

In [ ]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          exog               = data['temp'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 2217512.76


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Demanda y pronostico",
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()

# Funciones de peso
Es posible para las muestras aplicarles una ponderación. Por ejemplo podriamos modificar los fines de semana, que sabemos que tienen una menor demanda e incremantarlos un cierto porcentaje, para eliminar esa periodicidad.

> Esto es solo un ejemplo para mostrar como incorporar la función de pesos



In [ ]:
def weight_function(index):
    w = np.where((index.day_of_week == 1) | (index.day_of_week == 7), 1.3, 1.0)
    return w

In [ ]:
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24,                        # cuanto mira hacia atras
    weight_func=weight_function,    # funcion de peso
)

In [ ]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          exog               = data['temp'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 2215318.31


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Demanda y pronostico",
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()

# Uso de herramientas informacionales
La hipotesis planteada es la siguiente:

> La entropia permite utilizarse como peso para ajustar los valores la entropia de acuerdo a su "sorpresa".

Para esto instalaremos el paquete `ordpy` y usaremos sus funciones de entropía.


In [ ]:
!pip install ordpy

In [ ]:
import ordpy

In [ ]:
# generamos los pesos de entropia de acuerdo al avance durante el dia
entropia = [0] * 24
complejidad = [0] * 24
for i in range(24, len(data.demanda), 1):
    h, c = ordpy.complexity_entropy(data.demanda[i-24:i], dx=3)
    entropia.append(h)
    complejidad.append(c)
data['entropia'] = entropia
data['complejidad'] = complejidad

In [ ]:
# grafico lado a lado pronostico y valor real
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
trace1 = go.Scatter(x=data.index, y=data['demanda'], name="demanda", mode="lines")
trace2 = go.Scatter(x=data.index, y=data['entropia'], name="entropia", mode="lines")
trace3 = go.Scatter(x=data.index, y=data['complejidad'], name="complejidad", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2, secondary_y=True)
fig.add_trace(trace3, secondary_y=True)
fig.update_layout(
    title="Demanda y entropia",
    xaxis_title="Tiempo",
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Se crea la funcion de pesos
def weight_entropia(index):
    return data.loc[index, 'entropia']

In [ ]:
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24,                        # cuanto mira hacia atras
    weight_func=weight_entropia,    # funcion de peso
)

In [ ]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          exog               = data['temp'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 2216215.28


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Demanda y pronostico",
    xaxis_title="Tiempo",
    yaxis_title="Demanda",
)
fig.show()